<a href="https://colab.research.google.com/github/vamshikrishna1032000/CS612/blob/main/CS612_Term_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install biopython


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 30.2 MB/s eta 0:00:00


In [3]:
from Bio import SeqIO

fasta_file = "CSPD_IF1_20_id90.fas"

# Open the FASTA file
with open(fasta_file, "r") as file:
    # Parse the sequences
    for record in SeqIO.parse(file, "fasta"):
        # Access the sequence ID and sequence data
        sequence_id = record.id
        sequence_data = str(record.seq)
        
        # Do something with the sequence data
        print(f"Sequence ID: {sequence_id}")
        print(f"Sequence Data: {sequence_data}")


Sequence ID: null
Sequence Data: MEKGTVKWFNNAKGFGFICPEGGGEDIFAHYSTIQMDGYRTLKAGQSVQFDVHQGPKGNHASVIVPVEVEAAVAMAKEDNIEMQGTVLETLPNTMFRVELENGHVVTAHISGKMRKNYIRILTGDKVTVELTPYDLSKGRIVFRSR
Sequence ID: B595494_C4L949_C4L946_3
Sequence Data: MATGTVKWFNNAKGFGFICPDEGGDDIFAHYSTIDMNGYKTLKAGQPVEFEVQAGPKGSHATSIVP-T------MSKEDSIEMQGTILETLPNTMFRVELENGHVVTAHISGKMRKNYIRILTGDKVTVQLTPYDLSKGRIVFRSR
Sequence ID: B379066_C1A9Q1_C1A9Q0_1
Sequence Data: RTTGTVKWFNDAKGFGFITPSDGQKDCFVHHSAIQGNGFKSLTEGETVEFDIVQGQKGPAAENVTRS--------MKQDMIEFEGVVTDVLPSTMFRVKLENDHEVLSTVAGKMRKFRIRILAGDRVTVEVSPYDLSRGRITFRHK
Sequence ID: B879295_F3M1B2_F3M099_13
Sequence Data: MRTGIVKQFDSRSPYGFIEDDLTGASYFVFYKSIKESGYKRLEVGQRVRYQLAQGKKGLQCINVYNN-------MAKEDVIEVEGKVIDTLPNAMFKVELENGATILAHVSGKIRMHYIRILPGDRVTVELSPYDLTKGRITYRFK
Sequence ID: B563178_B8D9B2_B8D9A5_7
Sequence Data: -IKGQVKWFNESKGFGFITPSDGSKDVFVHFSSIQGNGFKTLTEGQNVEFEIQDGQKGPAAVNVFSL-------MSKEENIEMQGVVIDTLPNTMFRVELENKHIVTAHISGKMRKNYIRILTGDKVTVELTPYDLTKGRIIFRSR
Sequence ID: B937450_F1

In [81]:
import numpy as np
from Bio import SeqIO
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam
import os

def load_sequences_from_fasta(fasta_file):
    # Read sequences from the FASTA file
    sequences = []
    with open(fasta_file, "r") as file:
        for record in SeqIO.parse(file, "fasta"):
            sequences.append(str(record.seq))
    return sequences

def encode_sequences(sequences, amino_acids):
    # Create a mapping of amino acids to indices
    aa_to_index = {aa: i for i, aa in enumerate(amino_acids)}

    # Convert sequences to numerical representation
    encoded_matrix = np.zeros((len(sequences), len(sequences[0]), len(amino_acids)), dtype=int)
    for i, sequence in enumerate(sequences):
        for j, aa in enumerate(sequence):
            encoded_matrix[i, j, aa_to_index[aa]] = 1
    return encoded_matrix

# Define the path to the FASTA file and other parameters
fasta_file = "CSPD_IF1_20_id90.fas"
sequence_length = 146
latent_dim = 10
epochs = 600

# Check if the file exists
if not os.path.isfile(fasta_file):
    raise FileNotFoundError(f"File '{fasta_file}' not found.")

# Load sequences from the FASTA file
sequences = load_sequences_from_fasta(fasta_file)

# Split the data into train and validation sets
sequences_train, sequences_val = train_test_split(sequences, test_size=0.1)

# Define the mapping of amino acids to indices
amino_acids = "ACDEFGHIKLMNPQRSTVWY-"

# Encode sequences
encoded_train = encode_sequences(sequences_train, amino_acids)
encoded_val = encode_sequences(sequences_val, amino_acids)

# Define the autoencoder architecture
input_seq = Input(shape=(sequence_length, len(amino_acids)))
encoded = Dense(latent_dim, activation='relu')(input_seq)
decoded = Dense(len(amino_acids), activation='softmax')(encoded)
autoencoder = Model(input_seq, decoded)

# Compile the autoencoder
optimizer = Adam(learning_rate=0.001)
autoencoder.compile(optimizer=optimizer, loss='categorical_crossentropy')

# Train the autoencoder
autoencoder.fit(encoded_train, encoded_train,
                batch_size=32,
                epochs=epochs,
                shuffle=True,
                validation_data=(encoded_val, encoded_val))

# Reconstruct sequences from the autoencoder
reconstructed_sequences = autoencoder.predict(encoded_train)

# Convert numerical representation back to amino acid sequences
reconstructed_sequences = np.argmax(reconstructed_sequences, axis=2)
reconstructed_sequences = [''.join([amino_acids[index] for index in sequence]) for sequence in reconstructed_sequences]


Epoch 1/600
3/3 [==============================] - 1s 122ms/step - loss: 3.0368 - val_loss: 3.0313
Epoch 2/600
3/3 [==============================] - 0s 21ms/step - loss: 3.0276 - val_loss: 3.0223
Epoch 3/600
3/3 [==============================] - 0s 22ms/step - loss: 3.0185 - val_loss: 3.0133
Epoch 4/600
3/3 [==============================] - 0s 21ms/step - loss: 3.0093 - val_loss: 3.0042
Epoch 5/600
3/3 [==============================] - 0s 22ms/step - loss: 3.0002 - val_loss: 2.9951
Epoch 6/600
3/3 [==============================] - 0s 23ms/step - loss: 2.9909 - val_loss: 2.9860
Epoch 7/600
3/3 [==============================] - 0s 22ms/step - loss: 2.9817 - val_loss: 2.9769
Epoch 8/600
3/3 [==============================] - 0s 22ms/step - loss: 2.9725 - val_loss: 2.9677
Epoch 9/600
3/3 [==============================] - 0s 21ms/step - loss: 2.9632 - val_loss: 2.9586
Epoch 10/600
3/3 [==============================] - 0s 23ms/step - loss: 2.9540 - val_loss: 2.9494
Epoch 11/600
3/3 [

In [82]:
print("Encoded Matrix:", encoded_val)

Encoded Matrix: [[[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [1 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [1 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [1 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [1 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 ...

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [1 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 1]
  [0 0 0 ... 0 0 1]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]]


In [83]:
print("Reconstructed Sequences:", reconstructed_sequences)

Reconstructed Sequences: ['-ITGLVKWFNPEKGFGFITPKDGSKDVFVHFSAIQSNEFRTLNENQEVEFSVEQGPKGPSAVNVVAL-------MAKEDNIEMQGTVLETLPNTMFRVELENGHVVTAHISGKMRKNYIRILTGDKVTVELTPYDLSKGRIVFRSR', 'MKEGKVKWFNNAKGYGFINAKG-GDDLFVHFSTIEMEGYKTIKAGQKVIFDIIEGPKGQHAVNIRAVKD-----MSKEDSFEMEGTVVDTLPNTMFRVELENGHVVTAHISGKMRKNYIRILTGDKVRVDLTPYDLSKGRITYRAR', '--TGTVKWFNESKGFGFIEQESGP-DVFAHFSAISSTGFKTLTEGQKVQFTVTQGQKGPQAENIVAL-------MAKEDSIEMQGTIIDTLPNTMFRVELENGHIVTAHISGKMRKNYIRILTGDKVTVQLTPYDLSKGRIVFRAR', 'MATGRVKWFSNDRGFGFIEPDNGERELFAHHQNIIMEGYKTLKDFQRVTYDVEHGKNGRHAVNIIPGEA-----A-KEDHIEFEGEVIDTLPNTTFRVKLENDHVIIAHISGKMRKHYIRILTGDKVKVEMTPYDLTKGRITFRER', 'MATGTVKWFNNAKGFGFIDPEGGGEDIFAHYSTIQMEGYKTLKAGQAVNFELQQGPKGNHASVIVPNE------MAKEDSIEMQGTILETLPNTMFRVELENGHVVIAHISGKMRKNYIRILTGDKVTVALTPYDLSKGRIVFRSR', '-IKGQVKWFNESKGFGFITPADGSKDVFVHFSAIQGNGFKTLAEGQNVEFEIQDGQKGPSAVNVTAL-------MAKEDNIEMQGTVLDTLPNTMFRVELENGHVVTAHISGKMRKNYIRILTGDKVTVELTPYDLSKGRIVFRSR', 'MATGIVKWFNESKGFGFISPDEGGEDLFAHFSAIQSKGFKTLAEGQRVSFDVTTGPKGKQASNIQPA------

In [84]:
reconstructed_sequences[0]

'-ITGLVKWFNPEKGFGFITPKDGSKDVFVHFSAIQSNEFRTLNENQEVEFSVEQGPKGPSAVNVVAL-------MAKEDNIEMQGTVLETLPNTMFRVELENGHVVTAHISGKMRKNYIRILTGDKVTVELTPYDLSKGRIVFRSR'

In [75]:
# Epochs = 550
sequence_a = "MEKGTVKWFNNAKGFGFICPEGGGEDIFAHYSTIQMDGYRTLKAGQSVQFDVHQGPKGNHASVIVPVEVEAAVAMAKEDNIEMQGTVLETLPNTMFRVELENGHVVTAHISGKMRKNYIRILTGDKVTVELTPYDLSKGRIVFRSR"
sequence_b = reconstructed_sequences[0]
count = 0
for char_a, char_b in zip(sequence_a, sequence_b):
    if char_a != char_b:
        count=count+1
        #print(char_a)
count

51

In [85]:
# Epochs = 600
sequence_a = "MEKGTVKWFNNAKGFGFICPEGGGEDIFAHYSTIQMDGYRTLKAGQSVQFDVHQGPKGNHASVIVPVEVEAAVAMAKEDNIEMQGTVLETLPNTMFRVELENGHVVTAHISGKMRKNYIRILTGDKVTVELTPYDLSKGRIVFRSR"
sequence_b = reconstructed_sequences[0]
count = 0
for char_a, char_b in zip(sequence_a, sequence_b):
    if char_a != char_b:
        count=count+1
        #print(char_a)
count

40

In [70]:
# Epochs = 610
sequence_a = "MEKGTVKWFNNAKGFGFICPEGGGEDIFAHYSTIQMDGYRTLKAGQSVQFDVHQGPKGNHASVIVPVEVEAAVAMAKEDNIEMQGTVLETLPNTMFRVELENGHVVTAHISGKMRKNYIRILTGDKVTVELTPYDLSKGRIVFRSR"
sequence_b = reconstructed_sequences[0]
count = 0
for char_a, char_b in zip(sequence_a, sequence_b):
    if char_a != char_b:
        count=count+1
        #print(char_a)
count

0

In [86]:
# Measure Reconstruction Error

# 1. Learning Error (Loss)
loss = autoencoder.evaluate(encoded_val, encoded_val)
print("Reconstruction Loss:", loss)

# 2. Amino Acid Reconstruction Accuracy
def calculate_amino_acid_accuracy(original_sequences, reconstructed_sequences):
    total_positions = len(original_sequences[0])
    correct_positions = 0
    for original, reconstructed in zip(original_sequences, reconstructed_sequences):
        for char_original, char_reconstructed in zip(original, reconstructed):
            if char_original == char_reconstructed:
                correct_positions += 1
    accuracy = correct_positions / (len(original_sequences) * total_positions)
    return accuracy

accuracy = calculate_amino_acid_accuracy(sequences_train, reconstructed_sequences)
print("Amino Acid Reconstruction Accuracy:", accuracy)

# 3. Most Common Amino Acid Reconstruction
def calculate_most_common_aa_accuracy(original_sequences, reconstructed_sequences):
    total_positions = len(original_sequences[0])
    correct_positions = 0
    for original, reconstructed in zip(original_sequences, reconstructed_sequences):
        original_counts = {aa: original.count(aa) for aa in amino_acids}
        most_common_original = max(original_counts, key=original_counts.get)
        for char_original, char_reconstructed in zip(original, reconstructed):
            if char_original == most_common_original and char_reconstructed == most_common_original:
                correct_positions += 1
    accuracy = correct_positions / (len(original_sequences) * total_positions)
    return accuracy

accuracy = calculate_most_common_aa_accuracy(sequences_train, reconstructed_sequences)
print("Most Common Amino Acid Reconstruction Accuracy:", accuracy)


1/1 [==============================] - 0s 33ms/step - loss: 0.0293
Reconstruction Loss: 0.029293900355696678
Amino Acid Reconstruction Accuracy: 0.9980022831050228
Most Common Amino Acid Reconstruction Accuracy: 0.09888698630136987
